# Load and Preprocess Data

In [15]:
import pandas as pd

In [16]:
# Load datasets
fake_df = pd.read_csv("dataset/Fake.csv")
real_df = pd.read_csv("dataset/True.csv")

In [17]:
# Add labels
fake_df["label"] = 0  # Fake news
real_df["label"] = 1  # Real news

In [18]:
# Combine both datasets
df = pd.concat([fake_df, real_df]).sample(frac=1, random_state=42).reset_index(drop=True)

In [19]:
# Display dataset info
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB
None


In [20]:
df.head()

,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",0
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",1
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",1
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",0
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",1


In [10]:
import pandas as pd
import fasttext
import random
import os
from sklearn.metrics import classification_report

# Load Data
fake_df = pd.read_csv("dataset/Fake.csv")
true_df = pd.read_csv("dataset/True.csv")

# Add Labels
fake_df["label"] = "__label__0"  # Fake News → Label 0
true_df["label"] = "__label__1"  # True News → Label 1

# Combine & Shuffle Data
data = pd.concat([fake_df, true_df])
data = data.sample(frac=1, random_state=42).reset_index(drop=True)  # Shuffle

# Remove newlines & save FastText format
def save_fasttext_format(filename, df):
    with open(filename, "w", encoding="utf-8") as f:
        for _, row in df.iterrows():
            text = row["text"].replace("\n", " ").strip()  # Remove newlines
            f.write(f"{row['label']} {text}\n")

# Split into Train & Test (80-20)
split_ratio = 0.8
split_index = int(len(data) * split_ratio)

train_data = data[:split_index]
test_data = data[split_index:]

# Save Train & Test Data
save_fasttext_format("train.txt", train_data)
save_fasttext_format("test.txt", test_data)

# Train FastText Model
model = fasttext.train_supervised("train.txt", epoch=25, lr=0.5, wordNgrams=2, loss='softmax')

# Evaluate Model
def evaluate_model(model, test_file):
    y_true, y_pred = [], []
    
    with open(test_file, "r", encoding="utf-8") as f:
        for line in f:
            label, text = line.split(" ", 1)
            true_label = int(label.replace("__label__", ""))
            
            text = text.strip().replace("\n", " ")  # Ensure clean input
            predicted_label = int(model.predict(text)[0][0].replace("__label__", ""))
            
            y_true.append(true_label)
            y_pred.append(predicted_label)

    print(classification_report(y_true, y_pred))

# Run Evaluation
evaluate_model(model, "test.txt")

# Save Trained Model
model.save_model("best_fasttext_model.bin")

print("✅ Model Training & Evaluation Completed Successfully!")


Read 14M words
Number of words:  357369
Number of labels: 2
Progress: 100.0% words/sec/thread:  258510 lr:  0.000000 avg.loss:  0.009294 ETA:   0h 0m 0s


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4758
           1       1.00      1.00      1.00      4222

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980

✅ Model Training & Evaluation Completed Successfully!


In [ ]:
import fasttext

# Load trained model
model_path = "best_fasttext_model.bin"
try:
    model = fasttext.load_model("best_fasttext_model.bin")
    print(f"✅ Successfully loaded model: {model_path}")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    exit()

# Function for manual testing
def test_manual_input():
    while True:
        text = input("\n📝 Enter news text (or type 'exit' to quit): ")
        if text.lower() == "exit":
            print("🚪 Exiting manual testing...")
            break

        # Predict label
        label, probability = model.predict(text)

        # Ensure only two classes
        if label[0] == "__label__0":  # Fake news
            print(f"🔍 Prediction: ❌ FAKE NEWS (0) | Confidence: {probability[0]:.4f}")
        else:  # True news (since only 0 and 1 exist)
            print(f"🔍 Prediction: ✅ TRUE NEWS (1) | Confidence: {probability[0]:.4f}")

# Run testing
test_manual_input()


✅ Successfully loaded model: best_fasttext_model.bin



📝 Enter news text (or type 'exit' to quit):  A stunning story of corruption, cronyism and cover-ups and oh yeah, the Clinton s FBI investigators from across the country have been following leads into reports of bribery involving the Clinton Foundation. Multiple field offices have been involved in the investigation. A report in Sunday s Wall Street Journal (WSJ) by Devlin Barrett revealed that agents assigned to the New York field office have been carrying the bulk of the work in investigating the Clinton Foundation. They have received assistance from the FBI field office in Little Rock according to  people familiar with the matter, the WSJ reported. Other offices, including Los Angeles and Washington, D.C., have been collecting evidence regarding  financial crimes or influence-peddling. As far back as February 2016, FBI agents made the presentation to the Department of Justice (DOJ), the WSJ s sources stated.  The meeting didn t go well,  they wrote. While some sources said the FBI s 

🔍 Prediction: ❌ FAKE NEWS (0) | Confidence: 0.9997



📝 Enter news text (or type 'exit' to quit):  BRUSSELS (Reuters) - Belgian police detained four people over a foiled attack in 2015 on a high-speed train linking Brussels and Paris, federal prosecutors said on Monday. In August 2015 a machinegun-toting attacker wounded three people on a high-speed train before being overpowered by passengers, one in a series of militant attacks that has rocked Belgium and France over the past two years. Police carried out six house searches in Brussels and elsewhere in Belgium early on Monday, but did not find any explosives or weapons. A judge will decide at a later stage whether those detained could be kept further, prosecutors said. __label__0 Hillary s been using churches across America as a backdrop to promote the violent Black Lives Matter movement in an effort to pander for black votes. She proves every day how little respect she has for our religious institutions, the Christian community and more specifically, the Catholic church and it s  back

🔍 Prediction: ❌ FAKE NEWS (0) | Confidence: 0.7668



📝 Enter news text (or type 'exit' to quit):  WASHINGTON (Reuters) - The husband of White House adviser Kellyanne Conway criticized President Donald Trump’s tweets about the administration’s temporary travel ban on Monday, saying they were undercutting the Justice Department’s ability to defend his policies. The president’s messages “may make some people feel better,” but they will not help the administration achieve its goal of getting the Supreme Court to rule in its favor, George Conway said in a Twitter message.  George Conway is a lawyer who last week withdrew from contention for a senior Justice Department post; his wife was Trump’s 2016 campaign manager before joining him in the White House as a senior adviser who is one of Trump’s most vigorous defenders. In later Twitter messages, George Conway emphasized he still “very, very strongly” supports Trump and his executive order, but he said tweets on legal matters “seriously undermine” the administration’s agenda. He also said tha

🔍 Prediction: ✅ TRUE NEWS (1) | Confidence: 1.0000



📝 Enter news text (or type 'exit' to quit):  (Reuters) - Britain’s Sir Elton John will not perform at the inauguration of Donald Trump in Washington as U.S. president, the singer-songwriter’s spokeswoman said on Wednesday. Anthony Scaramucci, an economic adviser to Trump’s campaign, said John would be performing in a BBC radio interview during which Scaramucci also said the Republican president-elect had a pro-gay rights stance. “Elton John is going to be doing our concert on the Mall for the inauguration,” Scaramucci added in the interview for  “HARDtalk,” broadcast on Wednesday. “Elton will not be performing at Trump’s inaugural,” the singer’s publicist, Fran Curtis, responded. John, a prominent gay rights activist, performed at fundraisers for Democrat Hillary Clinton during the U.S. presidential election campaign. He also objected when Trump played his “Rocket Man” and “Tiny Dancer” songs without permission during rallies and campaign appearances earlier this year, saying that any

🔍 Prediction: ✅ TRUE NEWS (1) | Confidence: 1.0000


In [ ]:
A stunning story of corruption, cronyism and cover-ups and oh yeah, the Clinton s FBI investigators from across the country have been following leads into reports of bribery involving the Clinton Foundation. Multiple field offices have been involved in the investigation. A report in Sunday s Wall Street Journal (WSJ) by Devlin Barrett revealed that agents assigned to the New York field office have been carrying the bulk of the work in investigating the Clinton Foundation. They have received assistance from the FBI field office in Little Rock according to  people familiar with the matter, the WSJ reported. Other offices, including Los Angeles and Washington, D.C., have been collecting evidence regarding  financial crimes or influence-peddling. As far back as February 2016, FBI agents made the presentation to the Department of Justice (DOJ), the WSJ s sources stated.  The meeting didn t go well,  they wrote. While some sources said the FBI s evidence was not strong enough, others believed the DOJ had no intention from the start of going any further. Barrett wrote that the DOJ officials were  stern, icy and dismissive of the case. Barrett wrote,  That was one of the weirdest meetings I ve ever been to,  one participant told others afterward, according to people familiar with the matter. It appears there was rift between the FBI and the DOJ about how to move forward with the investigation. Agents in the Washington office were directed to focus on a separate issue related to the actions of former Virginia Governor and Clinton Foundation Board member Terry McAuliffe. Agents inside the FBI believed they could build a stronger case if the investigation of McAuliffe and the foundation were combined.FBI Deputy Director Andrew McCabe seemed to be caught in the middle of the fight between DOJ officials who appeared to want to slow down or shut down the investigation and FBI agents who were eager to pour more resources into the investigation.Barrett wrote,  Are you telling me that I need to shut down a validly predicated investigation?  Mr. McCabe asked, according to people familiar with the conversation. After a pause, the official replied,  Of course not,  these people said. Some of the WSJ sources told Barrett that a  stand down  order had been given to the FBI agents by McCabe. Others denied that no such order was given.The story gets more complicated when you factor in that McCabe s wife, Dr. Jill McCabe had received a $467,500 campaign contribution in 2015 for a state senate race from McAuliffe.For entire story: Breitbart